## Mutli-Layer Perceptron

In [1]:
import sklearn
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn import datasets
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import metrics
from matplotlib import pyplot as plt

In [2]:
col_names = "rain,hot temp,mild temp,high humidity,strong winds,playTennis"
my_tennis_data = np.array(
      [[0, 1, 0, 1, 0, 0],
       [0, 1, 0, 1, 1, 0],
       [1, 0, 1, 1, 0, 1],
       [1, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 1, 0],
       [0, 0, 1, 1, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [1, 0, 1, 0, 0, 1],
       [0, 0, 1, 0, 1, 1],
       [1, 0, 1, 1, 1, 0]])

df = pd.DataFrame(data=my_tennis_data, columns=col_names.split(","))

In [3]:
x = df.drop("playTennis", axis=1)
y = df["playTennis"]

In [4]:
# The science comes into choosing this parameters
mlp_clf = MLPClassifier(hidden_layer_sizes=(10,4,5), max_iter=2000)
mlp_clf.fit(x, y) # Uses the entire dataset instead of training set

MLPClassifier(hidden_layer_sizes=(10, 4, 5), max_iter=2000)

In [9]:
x.iloc[-2:,:]

,rain,hot temp,mild temp,high humidity,strong winds
8,0,0,1,0,1
9,1,0,1,1,1


In [10]:
print(mlp_clf.predict(x.iloc[-2:]))
print(y.values[-2:])

[1 0]
[1 0]


In [11]:
mlp_clf.coefs_

[array([[ 3.86209887e-02, -2.22219090e-01,  5.06123045e-01,
          4.20194930e-02,  4.64965300e-03,  1.74338188e-01,
         -6.90088145e-02, -1.12423097e-11,  9.21648015e-02,
         -5.82018497e-01],
        [-3.76334078e-01, -3.86796310e-01,  5.24689184e-01,
         -9.38608780e-02, -3.54166007e-01,  8.53478439e-01,
         -2.31527472e-01, -1.30392714e-02,  3.73695492e-01,
          6.23642905e-01],
        [ 3.59432100e-01,  5.61144012e-01, -4.47539313e-01,
          1.29128910e-01,  4.04223367e-01, -9.14925478e-02,
          3.53685080e-01, -1.43826730e-01,  3.64838104e-01,
          1.71881338e-01],
        [-3.46730094e-01, -1.57754612e-03,  7.80988446e-01,
          3.21261119e-01,  1.01790532e-01,  7.35455760e-01,
         -3.50393699e-01, -8.48097034e-09, -1.59913423e-04,
         -1.71941610e-01],
        [-7.68559938e-01,  3.47657114e-02,  3.68697907e-01,
         -2.78641881e-01, -8.76586128e-01,  8.21998653e-01,
         -5.12660219e-01, -4.20319401e-11,  2.887430

In [12]:
mlp_clf.best_loss_

0.014705988649852435

In [13]:
mlp_clf.n_layers_

5

In [14]:
mlp_clf.out_activation_

'logistic'

# Complex Dataset
Dataset has non-binary valyes and each features spans different ranges

In [19]:
iris = datasets.load_iris()
x = iris.data
y = iris.target
pd.DataFrame(x, columns=iris.feature_names).describe()


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


Now we will do a standard KFold evaluation of the mode instead of passing the features directly into the model, we first learn a scaling, apply that scaling, then pas it through to the mode. Then on evaluation we apply that scaling to the test set before making out test predictions

In [23]:
kfold = model_selection.KFold(5, shuffle=True, random_state=2)
prec, rec, f1 = [], [], []

mlp_clf = MLPClassifier(hidden_layer_sizes=(10,4,5), max_iter=6000, activation="relu", random_state=2)
x_scaler = preprocessing.MinMaxScaler() # preprocessing.StandardScaler()

for train_idx, test_idx in kfold.split(x):
    x_train, x_test =  x[train_idx], x[test_idx]
    y_train, y_test =  y[train_idx], y[test_idx]

    x_train = x_scaler.fit_transform(x_train)
    x_test = x_scaler.fit_transform(x_test)

    mlp_clf.fit(x_train, y_train)
    
    y_pred = mlp_clf.predict(x_test)
    
    print(y_pred)
    print(y_test)
    
    rec += [metrics.recall_score(y_pred, y_test, average="weighted")]
    prec += [metrics.precision_score(y_pred, y_test, average="weighted")]
    f1 += [metrics.f1_score(y_pred, y_test, average="weighted")]

print("recall", np.mean(rec), np.std(rec))
print("precision", np.mean(prec), np.std(prec))
print("f1", np.mean(f1), np.std(f1))

    

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 2 1 1 1 1 1 2 2 2 2 2 2 2 2]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2]
[0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2]
[0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2]
[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2]
[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2]
[0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 1 2 1 2 2 2 2 1 1 2 2 2 2 2]
[0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2]
[0 0 0 0 0 0 0 0 0 0 0 1 1 2 1 2 1 2 2 1 1 1 2 2 2 2 2 2 2 2]
[0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2]
recall 0.9400000000000001 0.06110100926607785
precision 0.9523951048951049 0.04918768560734927
f1 0.9403224400871458 0.06075861491243073
